# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#testingoutside)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U kubernetes==9.0.0
    !pip install -U azure

    !pip install -U git+https://github.com/yjb-ds/functions-demo.git
    !pip install -U git+https://github.com/mlrun/mlrun.git@development

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [1]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [2]:
%%nuclio cmd -c
pip install kfserving==0.2.0
pip install  numpy==1.16.4 tensorflow==2.0.0b1 pandas==0.25.3
pip install -U azure joblib
pip install -U git+https://github.com/yjb-ds/functions-demo.git
pip install -U git+https://github.com/mlrun/mlrun.git@development

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

from functions.models import classifier_gen
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class FeaturesEngineer(BaseEstimator, TransformerMixin):
    """Engineer features from raw input.

    A standard transformer mixin that can be inserted into a scikit learn Pipeline.
    
    To use, 
    >>> ffg = FeaturesEngineer()
    >>> ffg.fit(X)
    >>> x_transformed = ffg.transform(X)
    or
    >>> ffg = FeaturesEngineer()
    >>> x_transformed = ffg.fit_transform(X)
    
    In a pipeline:
    >>> from sklearn.pipeline import Pipeline
    >>> from sklearn.preprocessing import StandardScaler
    >>> transformers = [('feature_gen', FeaturesEngineerFeature()), 
                        ('scaler', StandardScaler())]
    >>> transformer_pipe = Pipeline(transformers)
    """
    def fit(self, X, y=None):
        """fit is unused here
        """
        return self

    def transform(self, X, y=None):
        """Transform raw input data as a preprocessing step.
        
        :param X: Raw input features
        
        Returns a DataFrame of features.
        """
        x = X.copy()
        
        # do some cool feature engineering:here we replace by a N(2,2) series
        m = 2.0
        s = 2.0
        
        print(x.shape) # transform only if x is 2D
        if len(x.shape) == 2:
            print('here')
            n, f = x.shape
            
            if type(x)==np.ndarray:
                x[:, f-1] = np.random.normal(m, s, n)
            else:
                x.values[:, f-1] = np.random.normal(m, s, n)
        
        x = x.astype('float')
        
        return x

class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        KubeFlow serving model wrapper.
        
        :param name:            model name
        :param model_dir: path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        ffg = FeaturesEngineer()

        # scaler
        ss = StandardScaler()
        ss.__dict__ = joblib.load(f"{self.model_dir}/scaler.pickle")

        # keras model
        ksm = classifier_gen()
        ksm.load_weights(f"{self.model_dir}/weights.h5")

        pipe = make_pipeline(ffg, ss, ksm)

        self.classifier = pipe

    def predict(self, body):
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
#             feats = np.asarray(body['instances'])
#             result: np.ndarray = self.classifier.predict(feats)
            return body
#             return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [4]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [5]:
from mlrun import new_model_server, mount_v3io

In [6]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "tfkeras_serving_v2"

In [7]:
fn = new_model_server(MODEL_NAME, 
                      models={"tfkeras_joblib_v2": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

In [8]:
#fn.verbose=True
addr = fn.deploy(project="refactoring-demos")

[mlrun] 2020-01-01 19:49:19,306 deploy started
[nuclio] 2020-01-01 19:49:23,428 (info) Build complete
[nuclio] 2020-01-01 19:49:30,540 done updating tfkeras-serving-v2, function address: 3.137.70.243:30007


## **test**

In [9]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [10]:
# Grab some data - balanced dataset
features = pd.read_csv("x_test_50.csv")
labels = pd.read_csv("y_test_50.csv")

In [15]:
event = {"instances" : features.values.tolist()}

<a id="testingoutside"></a>
##### **testing our model outside the server**

In [32]:
from sklearn.metrics  import accuracy_score

# create and load
model = MyKerasClassifier('anyname', model_dir=TARGET_PATH)
model.load()

y_pred = model.predict(event)
print(type(y_pred), ':', len(y_pred['instances']))

<class 'dict'> : 100


In [27]:
import urllib
import json
params = json.dumps(event).encode('utf8')
req = urllib.request.Request(addr + "/tfkeras_joblib_v2/predict", data=params, method='PUT', headers={'content-type': 'application/json'})
resp = urllib.request.urlopen(req)

In [31]:
resp.readlines()

[b'{"instances": []}']

In [30]:
dir(resp)


['__abstractmethods__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 '_check_close',
 '_close_conn',
 '_get_chunk_left',
 '_method',
 '_peek_chunked',
 '_read1_chunked',
 '_read_and_discard_trailer',
 '_read_next_chunk_size',
 '_read_status',
 '_readall_chunked',
 '_readinto_chunked',
 '_safe_read',
 '_safe_readinto',
 'begin',
 'chunk_left',
 'chunked',
 'close',
 'closed',
 'code',
 'debuglevel',
 'detach',
 'fileno',
 'flush',
 'fp',
 'getcode',
 'gethead

In [23]:
resp = requests.put(addr + "/tfkeras_joblib_v2/predict", json=event)

print(resp.content)

b'{"instances": []}'


In [14]:
requests?

Type:        module
String form: <module 'requests' from '/conda/lib/python3.6/site-packages/requests/__init__.py'>
File:        /conda/lib/python3.6/site-packages/requests/__init__.py
Docstring:  
Requests HTTP Library
~~~~~~~~~~~~~~~~~~~~~

Requests is an HTTP library, written in Python, for human beings. Basic GET
usage:

   >>> import requests
   >>> r = requests.get('https://www.python.org')
   >>> r.status_code
   200
   >>> 'Python is a programming language' in r.content
   True

... or POST:

   >>> payload = dict(key1='value1', key2='value2')
   >>> r = requests.post('https://httpbin.org/post', data=payload)
   >>> print(r.text)
   {
     ...
     "form": {
       "key2": "value2",
       "key1": "value1"
     },
     ...
   }

The other HTTP methods are supported - see `requests.api`. Full documentation
is at <http://python-requests.org>.

:copyright: (c) 2017 by Kenneth Reitz.
:license: Apache 2.0, see LICENSE for more details.
